In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import os
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
FLAGS = None

def generate_embeddings():
    # Import data
    mnist = input_data.read_data_sets(FLAGS.data_dir,
                                      one_hot=True,
                                      fake_data=FLAGS.fake_data)
    sess = tf.InteractiveSession()

    # Input set for Embedded TensorBoard visualization
    # Performed with cpu to conserve memory and processing power
    with tf.device("/cpu:0"):
        embedding = tf.Variable(tf.stack(mnist.test.images[:FLAGS.max_steps], axis=0), trainable=False, name='embedding')

    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(FLAGS.log_dir + '/projector', sess.graph)

    # Add embedding tensorboard visualization. Need tensorflow version
    # >= 0.12.0RC0
    config = projector.ProjectorConfig()
    embed= config.embeddings.add()
    embed.tensor_name = 'embedding:0'
    embed.metadata_path = os.path.join(FLAGS.log_dir + '/projector/metadata.tsv')
    embed.sprite.image_path = os.path.join(FLAGS.data_dir + '/mnist_10k_sprite.png')

    # Specify the width and height of a single thumbnail.
    embed.sprite.single_image_dim.extend([28, 28])
    projector.visualize_embeddings(writer, config)

    saver.save(sess, os.path.join(
        FLAGS.log_dir, 'projector/a_model.ckpt'), global_step=FLAGS.max_steps)

def generate_metadata_file():
    # Import data
    mnist = input_data.read_data_sets(FLAGS.data_dir,
                                      one_hot=True,
                                      fake_data=FLAGS.fake_data)
    def save_metadata(file):
        with open(file, 'w') as f:
            for i in range(FLAGS.max_steps):
                c = np.nonzero(mnist.test.labels[::1])[1:][0][i]
                f.write('{}\n'.format(c))

    save_metadata(FLAGS.log_dir + '/projector/metadata.tsv')

def main(_):
    if tf.gfile.Exists(FLAGS.log_dir + '/projector'):
        tf.gfile.DeleteRecursively(FLAGS.log_dir + '/projector')
        tf.gfile.MkDir(FLAGS.log_dir + '/projector')
    tf.gfile.MakeDirs(FLAGS.log_dir  + '/projector') # fix the directory to be created
    generate_metadata_file()
    generate_embeddings()

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--fake_data', nargs='?', const=True, type=bool,
                        default=False,
                        help='If true, uses fake data for unit testing.')
    parser.add_argument('--max_steps', type=int, default=10000,
                        help='Number of steps to run trainer.')
    parser.add_argument('--data_dir', type=str, default='/Users/norman/Documents/workspace/mnist-tensorboard-embeddings/mnist_data',
                        help='Directory for storing input data')
    parser.add_argument('--log_dir', type=str, default='/Users/norman/Documents/workspace/mnist-tensorboard-embeddings/logs',
                        help='Summaries log directory')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)


In [3]:
# -*- coding: utf-8 -*-

import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np

PATH = '/disk1/embedding/'

LOG_DIR = PATH + 'mnist-tensorboard/log-1'
metadata = os.path.join(LOG_DIR, 'metadata.tsv')

images = tf.Variable(cell, name='images')
#def save_metadata(file):
with open(metadata, 'w') as metadata_file:
    for row in range(10000):
        c = 0
        metadata_file.write('{}\n'.format(c))



with tf.Session() as sess:
    saver = tf.train.Saver([images])
    
    sess.run(images.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images.ckpt'))

    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = images.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = metadata
    embedding.sprite.image_path = '/disk1/cell_work/0716/test.png'
    embedding.sprite.single_image_dim.extend([32, 32])
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)
    
# for calling the tensorboard you should be in that drive and call the entire path
#tensorboard --logdir=/Technical_works/tensorflow/mnist-tensorboard/log-1 --port=6006

In [ ]:
c

In [1]:
from multiprocessing import Pool as ThreadPool
import os 
import numpy as np

pool = ThreadPool(12) 
root_dir = '/disk1/cell_segment_save/'
npyList = os.listdir(root_dir)
npyList = [root_dir+n for n in npyList]
result = pool.map(np.load, npyList)
result = np.concatenate(result)

'''
from imgaug import augmenters as iaa

result_list = [result] 
seq = iaa.Sequential([
    iaa.Flipud(1), 
    #iaa.ContrastNormalization((1.40,1.60)),
])
seq2= iaa.Sequential([
    iaa.Fliplr(1), 
    #iaa.ContrastNormalization((1.40,1.60)),
])
seq3 = iaa.Sequential([
    iaa.Flipud(1), 
    iaa.Fliplr(1), 
    #iaa.ContrastNormalization((1.40,1.60)),
])

result_list.append(seq.augment_images(result))
result_list.append(seq2.augment_images(result))
result_list.append(seq3.augment_images(result))
result = np.concatenate(result_list,axis=0)
print(result.shape)
'''
print(result[:,:,:,0].shape)
#X = np.asarray([x.transpose((2,0,1)) for x in result])
#X = X.astype(np.float32)/(255.0/2) - 1.0

(6584, 32, 32)


In [2]:
cell= result.reshape(6584,-1)

In [ ]:
cell.shape

In [ ]:
import torchvision
import torch

X = np.asarray([x.transpose((2,0,1)) for x in result])
X = X.astype(np.float32)/(255.0/2) - 1.0
torchvision.utils.save_image(torch.from_numpy(X), 'test.png', normalize=True, padding=0, nrow=82)